<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong>
        Importando bibliotecas
    </strong>
</p>

In [1]:
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.styles import numbers
import xlwings as xw

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/><br/><br/><br/><br/>
        Lendo base de dados
    </strong>
</p>

In [2]:
def ler_arquivo(nome_arquivo):
    try:
       return pd.read_excel(f'bases/{nome_arquivo}.xlsx')
    except FileNotFoundError as e:
        return f'Erro: {e}'


<p style="font-size: 1.1rem; color: #9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/><br/><br/><br/><br/>
        Tratamento - Dados ausentes        
    </strong>
</p>

In [3]:
df = ler_arquivo('dados de entregas')


<p style="font-size: 1.1rem; color: #9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/><br/><br/><br/><br/>
        SLA    
    </strong>
</p>

In [4]:
#total de entregas
df_total_entregas = df.loc[ df['status'] == 'ENTREGUE']
df_total_entregas

#Total de entregas sem atraso
sla_ok = df_total_entregas.loc[ df['dias_atraso'] == 0].shape[0]
sla_ok

#Percentual SLA
sla_percentual = (sla_ok / df_total_entregas.shape[0]) * 100
f'{round(sla_percentual,2)}% das entregas foram feitas dentro do prazo.'

'37.24% das entregas foram feitas dentro do prazo.'

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/><br/><br/><br/><br/>
        KPI - Entragas atrasadas
    </strong>
</p>

In [5]:
# Total de entregas
total_entregas = df_total_entregas.shape[0]

# Total de entregas com atraso
entregas_atraso = df_total_entregas.loc[df_total_entregas['dias_atraso'] > 0].shape[0]

# Percentual de atraso
percentual_atraso = (entregas_atraso / total_entregas) * 100

print(f"{round(percentual_atraso, 1)}% das entregas tiveram atraso.")


62.8% das entregas tiveram atraso.


In [9]:
# Validando palavras chave sem trabalhar com dataframe (tabela)
desc = 'python é bom demais, a linguagem mais tranquila que existe'

palavras = desc.split()
try:
    palavras.index('python')
except ValueError as e:
    print (e)


Padronização da descrição

In [12]:
df['descricao_entrega'] = (
    df['descricao_entrega']
    .str.lower()
    .str.strip()

)

Separar as entregas atrasadas

In [16]:
df_atrasos = df.loc[ df['dias_atraso'] > 0 ].copy()


(3733, 12)

Categorizando as causas dos atrasos

In [22]:
df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains('chuva|temporal' , na=False),
    "causa_atraso"
] = "Clima"

df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains('operacional|rota|problema' , na=False),
    "causa_atraso"
] = "Problema Operacional"

df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains('extravio' , na=False),
    "causa_atraso"
] = "Extravio"

df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains('cliente' , na=False),
    "causa_atraso"
] = "Cliente Ausente"

df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains('normal' , na=False),
    "causa_atraso"
] = "Entrega Normal"

df_atrasos['causa_atraso'] = df_atrasos['causa_atraso'].fillna("Outros")

df_atrasos['causa_atraso'].head(10)

0            Extravio
2     Cliente Ausente
3     Cliente Ausente
5               Clima
6              Outros
7      Entrega Normal
9            Extravio
12             Outros
13             Outros
14     Entrega Normal
Name: causa_atraso, dtype: str

Principais causas de atrasos 

In [33]:
round(df_atrasos['causa_atraso'].value_counts(normalize=True) *100 , 1).reset_index(name='%')

,causa_atraso,%
0,Outros,36.8
1,Extravio,13.4
2,Entrega Normal,12.8
3,Cliente Ausente,12.7
4,Clima,12.7
5,Problema Operacional,11.7


Filial por causa de atraso

In [ ]:
causas_por_filiais = df_atrasos.groupby(['filial' , 'causa_atraso']).size().reset_index(name='Qtd').sort_values(['filial', 'Qtd'], ascending=[True, False])

causas_por_filiais

,filial,causa_atraso,Qtd
4,MG,Outros,201
1,MG,Clima,84
0,MG,Cliente Ausente,83
3,MG,Extravio,76
2,MG,Entrega Normal,73
5,MG,Problema Operacional,61
10,PR,Outros,242
7,PR,Clima,86
8,PR,Entrega Normal,84
11,PR,Problema Operacional,79


In [58]:
causas_por_filiais = df_atrasos.groupby(['filial' , 'causa_atraso']).size().reset_index(name='Qtd').sort_values(['filial', 'Qtd'], ascending=[True, False])

causas_por_filiais

,filial,causa_atraso,Qtd
4,MG,Outros,201
1,MG,Clima,84
0,MG,Cliente Ausente,83
3,MG,Extravio,76
2,MG,Entrega Normal,73
5,MG,Problema Operacional,61
10,PR,Outros,242
7,PR,Clima,86
8,PR,Entrega Normal,84
11,PR,Problema Operacional,79


In [63]:
causas_por_filiais = (
    df_atrasos
      .groupby(['filial', 'causa_atraso'])
      .size()
      .reset_index(name='Qtd')
)

total_filial = causas_por_filiais.groupby('filial')['Qtd'].transform('sum')

causas_por_filiais['%'] = (causas_por_filiais['Qtd'] / total_filial * 100).round(1)

causas_por_filiais = causas_por_filiais.sort_values(['filial', '%'], ascending=[True, False])

causas_por_filiais


,filial,causa_atraso,Qtd,%
4,MG,Outros,201,34.8
1,MG,Clima,84,14.5
0,MG,Cliente Ausente,83,14.4
3,MG,Extravio,76,13.1
2,MG,Entrega Normal,73,12.6
5,MG,Problema Operacional,61,10.6
10,PR,Outros,242,38.2
7,PR,Clima,86,13.6
8,PR,Entrega Normal,84,13.2
11,PR,Problema Operacional,79,12.5


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/><br/><br/><br/><br/>
        Tratamento - Tipagem correta para ID
    </strong>
</p>

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/><br/><br/><br/><br/>
        Tratamento - Padronização e Normalização
    </strong>
</p>